In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(0)
%matplotlib inline

In [2]:
def load_data(filename= "./advertising.csv"):
    data = np.genfromtxt(filename, dtype= None, delimiter= ",", skip_header= True)
    feature_X = data[:, : 3]
    print(feature_X.shape)
    sales_Y = data[:, 3]
    print(sales_Y.shape)
    feature_X_b = np.hstack((np.ones((sales_Y.shape[0], 1)), feature_X))

    return feature_X_b, sales_Y

In [3]:
X, y = load_data()

(200, 3)
(200,)


In [4]:
def create_individual(n= 4, bound= 10):
    individual= []
    
    for _ in range(0, n, 1):
        gene = random.uniform(-bound / 2, bound / 2)
        individual.append(gene)
        
    return individual 

In [5]:
individual = create_individual()
print(individual)

[3.4442185152504816, 2.5795440294030243, -0.79428419169155, -2.4108324970703663]


In [6]:
def compute_loss(individual):
    theta = np.array(individual)
    y_hat = X.dot(theta)    
    loss = np.multiply((y_hat - y), (y_hat - y)).mean()
    
    return loss

def compute_fitness(individual):
    loss = compute_loss(individual)
    fitness_value = 0
    
    fitness_value = 1 / (loss + 1)
    
    return fitness_value

In [7]:
individual = [4.09, 4.82, 3.10, 4.02]
fitness_score = compute_fitness(individual)
print(fitness_score)

1.0185991537088997e-06


In [8]:
def crossover(individual1, individual2, crossover_rate= 0.9):
    individual1_new = individual1.copy()
    individual2_new = individual2.copy()
    
    if random.random() < crossover_rate:
        point = random.randint(1, len(individual1) - 1)
        # individual1_new = individual1_new[point:] + individual2_new[point:]
        # individual2_new = individual2_new[point:] + individual1_new[point:]
        
        print(f"Crossover occurred at point {point}")
        individual1_new[point:], individual2_new[point:] = individual2_new[point:], individual1_new[point:]
    else:
        print("No crossover occurred")

        
    return individual1_new, individual2_new

In [9]:
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.41]
individual1_new, individual2_new = crossover(individual1, individual2, 0.5)
print("individual1:", individual1_new)
print("individual2:", individual2_new)

No crossover occurred
individual1: [4.09, 4.82, 3.1, 4.02]
individual2: [3.44, 2.57, -0.79, -2.41]


In [10]:
def mutation(individual, mutation_rate= 0.05):
    individual_m = individual.copy()
    
    for i in individual:
        if random.random() < mutation_rate:
            mutation = random.uniform(-0.5, 0.5)
            i += mutation
            
    return individual_m

In [11]:
before_mutation = [4.09, 4.82, 3.10, 4.02]
after_mutation = mutation(before_mutation, mutation_rate= 2.0)

print(before_mutation == after_mutation)

True


In [12]:
def initialize_population(m):
    population = [create_individual() for _ in range(m)]
    return population

In [13]:
def selection(sorted_old_population, m= 100):
    index1 = random.randint(0, m - 1)
    while True:
        index2 = random.randint(0, m - 1)
        if (index2 != index1):
            break
    
    individual_s = sorted_old_population[index1]
    if index2 > index1:
        individual_s = sorted_old_population[index2]
        
    return individual_s

In [14]:
def create_new_population(old_population, elitism= 2, gen= 1):
    m = len(old_population)
    sorted_population = sorted(old_population, key= compute_fitness)
    
    if gen % 1 == 0:
        print("Best loss: ", compute_loss(sorted_population[m - 1]), " with chromsome: ", sorted_population[m - 1])
        
    new_population = []
    while len(new_population) < m-elitism:
        # Selection
        individual1_s = selection(sorted_population)
        individual2_s = selection(sorted_population)
        
        # Crossover
        individual1_new, individual2_new = crossover(individual1_s, individual2_s)
        
        # Mutation
        individual1_new_m = mutation(individual1_new)
        individual2_new_m = mutation(individual2_new)
        
        new_population.extend([individual1_new_m, individual2_new_m])
        
        
    # Ensure the population size remains constant
    new_population = new_population[:m - elitism]
    
    # Copy elitism chromosomes that have best fitness score to the next generation
    for ind in sorted_population[:elitism]:
        new_population.append(ind.copy())
    
    return new_population, compute_loss(sorted_population[m - 1])

In [15]:
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.41]

old_population = [individual1, individual2]
create_new_population, _ = create_new_population(old_population)

Best loss:  123415.051528805  with chromsome:  [3.44, 2.57, -0.79, -2.41]


In [16]:
def run_GA():
    n_generations = 100
    m = 600
    population = initialize_population(m)
    losses_list = []
    for i in range(n_generations):
        # Calculate fitness scores
        fitness_scores = [compute_fitness(individual) for individual in population]
        
        # Find the best individual
        best_individual = population[np.argmax(fitness_scores)]
        best_loss = compute_loss(best_individual)
        losses_list.append(best_loss)
        
        # Create new population
        population, gen_best_loss = create_new_population(population, elitism=2, gen=i+1)
        
        if i % 10 == 0:  # Print progress every 10 generations
            print(f"Generation {i}: Best Loss = {gen_best_loss}")
    
    # Find the overall best individual
    final_fitness_scores = [compute_fitness(individual) for individual in population]
    overall_best_individual = population[np.argmax(final_fitness_scores)]
    overall_best_loss = compute_loss(overall_best_individual)
    
    print(f"\nFinal Best Loss: {overall_best_loss}")
    print(f"Final Best Individual: {overall_best_individual}")
    
    return losses_list, overall_best_individual

In [17]:
def visualize_loss(losses_list):
    plt.figure(figsize= (10, 6))
    plt.plot(range(len(losses_list), losses_list))
    plt.title("Best Loss Value per Generation")
    plt.xlabel("Generation")
    plt.ylabel("Loss value")
    plt.grid(True)
    plt.show()

In [18]:
losses_list, _ = run_GA()
visualize_loss(losses_list)

TypeError: 'list' object is not callable